In [101]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from decimal import Decimal
from datetime import datetime
import sys
sys.path.append('../')
import data
from thetadata_utils import curl
import importlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
from thetadata import ThetaClient
from credentials import USERNAME, PASSWORD
client = ThetaClient(username=USERNAME, passwd=PASSWORD, timeout=10)

If you require API support, feel free to join our discord server! http://discord.thetadata.us


[07-09-2024 23:59:40] INFO: Starting Theta Terminal v1.6.1 Revision C...
[07-09-2024 23:59:40] INFO: Config dir: /home/vscode/ThetaData/ThetaTerminal
[07-09-2024 23:59:41] INFO: [MDDS] Attempting login as awattscurnow@gmail.com
[07-09-2024 23:59:41] INFO: [FPSS] Attempting login as awattscurnow@gmail.com
[07-09-2024 23:59:42] INFO: [MDDS] CONNECTED: [nj-a.thetadata.us:12000], Bundle: STOCK.FREE, OPTION.STANDARD, INDEX.FREE
[07-09-2024 23:59:42] INFO: [FPSS] CONNECTED: [nj-a.thetadata.us:20000], Bundle: STOCK.FREE, OPTION.STANDARD, INDEX.FREE
[07-10-2024 00:53:18] INFO: [FPSS] Connection lost... Attempting to reconnect every 1 seconds
[07-10-2024 00:53:19] INFO: [FPSS] Attempting login as awattscurnow@gmail.com
[07-10-2024 00:53:54] INFO: [FPSS] CONNECTED: [nj-a.thetadata.us:20000], Bundle: STOCK.FREE, OPTION.STANDARD, INDEX.FREE
[07-10-2024 00:54:16] INFO: [MDDS] Connection lost... Attempting to reconnect every 1 seconds
[07-10-2024 00:54:16] INFO: [MDDS] Attempting login as awattscurn

In [78]:
config = {
    "underlying_ticker": 'spx',
    "options_ticker": 'spxw',
    "ivl": 1000 * 60,
    "ivl_name": '1m',
}
date = datetime(2024, 6, 26)
dateStr = date.strftime("%Y%m%d")

#### Expirations

In [ ]:
path = '/data/thetadata/options'
expirations = pd.read_csv(f"{path}/{config['options_ticker']}/expirations.csv.gz", compression='gzip')
expirations['date'] = pd.to_datetime(expirations['date'], format='%Y-%m-%d')
expirations

In [80]:
expiration = data.get_expiration(config['options_ticker'], dateStr)
expiration

'20240626'

#### Strikes

In [ ]:
# sd_strikes = data.get_option_sd_strikes(config['underlying_ticker'], config['options_ticker'], dateStr)
sd_strikes

#### Get historical call quotes for single strike

In [85]:
# Intraday quotes for single strike
atm_strike = data.get_option_atm_strike(config['underlying_ticker'], config['options_ticker'], dateStr)
# atm_strike += 5000 # For testing.  Passed
query = {"root": "SPXW", "start_date": expiration, "end_date": expiration, "ivl":config['ivl'], "exp": expiration, "strike": atm_strike, "right": "C"}
quotes = curl('hist/option/quote', query)
quotes['time'] = pd.to_datetime(quotes['ms_of_day'], unit='ms').dt.time
quotes

atm strike: 5460000
http://127.0.0.1:25510/v2/hist/option/quote?root=SPXW&start_date=20240626&end_date=20240626&ivl=60000&exp=20240626&strike=5465000&right=C&use_csv=true


,ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,time
0,34200000,0,5,0.0,50,0,5,0.0,50,20240626,09:30:00
1,34260000,2,5,6.4,50,46,5,6.5,50,20240626,09:31:00
2,34320000,113,5,5.6,50,10,5,5.8,50,20240626,09:32:00
3,34380000,6,5,6.1,50,1,5,6.2,50,20240626,09:33:00
4,34440000,79,5,6.1,50,126,5,6.3,50,20240626,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...
386,57360000,9,5,15.8,50,10,5,16.6,50,20240626,15:56:00
387,57420000,9,5,17.3,50,9,5,17.9,50,20240626,15:57:00
388,57480000,10,5,15.7,50,20,5,16.6,50,20240626,15:58:00
389,57540000,10,5,15.2,50,20,5,16.2,50,20240626,15:59:00


#### Bulk quotes

In [125]:
import pandas as pd

# Constants and setup
ivl = 1000  # 1 second
batch = 10 * 60 * 1000 # 10 minutes in milliseconds

start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 10)

start_time = 34200000  # 09:30 in milliseconds
# end_time = start_time + 1 * batch
end_time = 57600000    # 16:00 in milliseconds

# Iterate over the expiration dates
filtered_expirations = expirations[(expirations['date'] >= start_date) & (expirations['date'] <= end_date)]
for index, row in filtered_expirations.iterrows():
	expiration = row['date'].strftime('%Y%m%d')
	sd_strikes = data.get_option_sd_strikes(config['underlying_ticker'], config['options_ticker'], expiration)
	daily_data = pd.DataFrame()  # Initialize an empty DataFrame for daily data

	# Loop through between start_time and end_time
	current_time = start_time
	while current_time <= end_time:
		next_time = current_time + batch

		# Query for each block
		query = {
			"root": "SPXW",
			"start_date": expiration,
			"end_date": expiration,
			"start_time": current_time,
			"end_time": next_time,
			"ivl": ivl,
			"exp": expiration
		}
		batch_data = curl('bulk_hist/option/quote', query)
		print(f"Retrieved data for {expiration} from {current_time} to {next_time}.")

		# Only keep in 1SD strikes
		# Append 1SD batch data to the daily DataFrame
		batch_data = batch_data[batch_data['strike'].isin(sd_strikes['strike'])]
		daily_data = pd.concat([daily_data, batch_data], ignore_index=True)

		current_time = next_time

	# After collecting all hourly data for the day, write to a single file
	path = '/data/thetadata/options/spxw/0dte/1s/'
	filename = f'{expiration}.csv.gz'
	daily_data = quotes.drop_duplicates()
	daily_data.to_csv(path + filename, compression='gzip')
	print(f"Data for {expiration} saved to {filename}.")

atm strike: 4860000
Retrieved data for 20240201 from 34200000 to 34800000.
Retrieved data for 20240201 from 34800000 to 35400000.
Retrieved data for 20240201 from 35400000 to 36000000.
Retrieved data for 20240201 from 36000000 to 36600000.
Retrieved data for 20240201 from 36600000 to 37200000.
Retrieved data for 20240201 from 37200000 to 37800000.
Retrieved data for 20240201 from 37800000 to 38400000.
Retrieved data for 20240201 from 38400000 to 39000000.
Retrieved data for 20240201 from 39000000 to 39600000.
Retrieved data for 20240201 from 39600000 to 40200000.
Retrieved data for 20240201 from 40200000 to 40800000.
Retrieved data for 20240201 from 40800000 to 41400000.
Retrieved data for 20240201 from 41400000 to 42000000.
Retrieved data for 20240201 from 42000000 to 42600000.
Retrieved data for 20240201 from 42600000 to 43200000.
Retrieved data for 20240201 from 43200000 to 43800000.
Retrieved data for 20240201 from 43800000 to 44400000.
Retrieved data for 20240201 from 44400000 to 

#### Restore test

In [111]:
test = pd.read_csv('/data/thetadata/options/spxw/0dte/1s/20240102.csv.gz', compression='gzip')
test
print(test['strike'].unique())

[1200000 1400000 1600000 1800000 2000000 2200000 2400000 2600000 2800000
 3000000 3200000 3400000 3500000 3600000 3650000 3700000 3750000 3800000
 3850000 3900000 3950000 4000000 4050000 4075000 4100000 4125000 4150000
 4175000 4200000 4225000 4250000 4275000 4300000 4325000 4330000 4340000
 4350000 4360000 4370000 4375000 4380000 4390000 4400000 4410000 4415000
 4420000 4425000 4430000 4435000 4440000 4445000 4450000 4455000 4460000
 4465000 4470000 4475000 4480000 4485000 4490000 4495000 4500000 4505000
 4510000 4515000 4520000 4525000 4530000 4535000 4540000 4545000 4550000
 4555000 4560000 4565000 4570000 4575000 4580000 4585000 4590000 4595000
 4600000 4605000 4610000 4615000 4620000 4625000 4630000 4635000 4640000
 4645000 4650000 4655000 4660000 4665000 4670000 4675000 4680000 4685000
 4690000 4695000 4700000 4705000 4710000 4715000 4720000 4725000 4730000
 4735000 4740000 4745000 4750000 4755000 4760000 4765000 4770000 4775000
 4780000 4785000 4790000 4795000 4800000 4805000 48